# Facebook Advantage+ 대량 광고 생성기

**Excel 파일에서 카피를 읽어와 자동으로 광고를 생성합니다.**

## 사용법
1. `fb_ad_config.xlsx`에서 **노란색 셀** 채우기 (본문, 헤드라인, 설명, URL, CTA)
2. 저장
3. 이 노트북 실행

---\n## STEP 1: 패키지 설치 & 로드

In [ ]:
!pip3 install facebook-business openpyxl

In [ ]:
import os, time, json
from pathlib import Path
from collections import defaultdict
import openpyxl

from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adimage import AdImage
from facebook_business.adobjects.advideo import AdVideo

---
## STEP 2: 설정값 입력

⚠️ **실제 값으로 교체하세요!**

In [ ]:
# ============================================================
# ⚠️ Meta API 설정
# ============================================================
APP_ID           = "YOUR_APP_ID"
APP_SECRET       = "YOUR_APP_SECRET"
ACCESS_TOKEN     = "YOUR_ACCESS_TOKEN"
AD_ACCOUNT_ID    = "act_XXXXXXXXX"
PAGE_ID          = "YOUR_PAGE_ID"
PIXEL_ID         = "YOUR_PIXEL_ID"
INSTAGRAM_ID     = ""                       # 선택사항

# ⚠️ 경로 설정
ADS_BASE_FOLDER  = "YOUR_PARTH"
XLSX_PATH        = "YOUR_PARTH"

# 캠페인 설정
CAMPAIGN_OBJ     = "OUTCOME_SALES"
DAILY_BUDGET     = 77777          # 일 예산 (원)
BID_STRATEGY     = "LOWEST_COST_WITHOUT_CAP"
OPT_GOAL         = "OFFSITE_CONVERSIONS"


---
## STEP 3: Excel에서 광고 설정 읽기

Excel 파일에서 작성한 카피를 읽어옵니다.

In [ ]:
# Excel 읽기
def load_xlsx(path):
    wb = openpyxl.load_workbook(path, data_only=True)
    ws = wb["광고 설정"]
    headers = [cell.value for cell in ws[1]]
    ads = []
    for row in ws.iter_rows(min_row=2, values_only=True):
        if row[0] is None:
            break
        ads.append(dict(zip(headers, row)))
    wb.close()
    return ads

ad_list = load_xlsx(XLSX_PATH)
print(f"✅ {len(ad_list)}개 광고 설정 로드 완료")
print()

# 입력 확인
ok, skip = 0, 0
for row in ad_list:
    no = row["No"]
    pt = str(row.get("본문 텍스트\n(primary_text)", "") or "").strip()
    hl = str(row.get("헤드라인\n(headline)", "") or "").strip()
    url = str(row.get("랜딩 URL\n(link_url)", "") or "").strip()
    
    if pt and hl and url:
        ok += 1
    else:
        skip += 1
        print(f"  ⚠️ #{no} {row['광고명']} → 카피 미입력 (건너뜀)")

print(f"\n📊 입력 완료: {ok}개 / 미입력: {skip}개")


---
## STEP 4: 소재 파일 매핑 확인

In [ ]:
# 폴더별 소재 파일 확인
def collect_media(base, folder_name):
    folder = Path(base) / folder_name
    if not folder.exists():
        return [], []
    images = sorted([str(f) for f in (folder/"Img").iterdir() if f.suffix.lower() in {".jpg",".jpeg",".png"}]) if (folder/"Img").exists() else []
    videos = sorted([str(f) for f in (folder/"Video").iterdir() if f.suffix.lower() in {".mp4",".mov"}]) if (folder/"Video").exists() else []
    return images, videos

total_img, total_vid = 0, 0
for row in ad_list:
    imgs, vids = collect_media(ADS_BASE_FOLDER, row["소재 폴더"])
    total_img += len(imgs)
    total_vid += len(vids)
    status = "✅" if (imgs or vids) else "❌"
    print(f"  {status} {row['소재 폴더']}: 이미지 {len(imgs)}개, 영상 {len(vids)}개")

print(f"\n📊 총 소재: 이미지 {total_img}개 + 영상 {total_vid}개 = {total_img+total_vid}개 크리에이티브")

---
## STEP 5: 핵심 함수 (수정 불필요)

In [ ]:
def init_api():
    FacebookAdsApi.init(APP_ID, APP_SECRET, ACCESS_TOKEN)
    account = AdAccount(AD_ACCOUNT_ID)
    info = account.api_get(fields=["name", "currency", "timezone_name"])
    print(f"✅ API 연결: {info['name']} ({info['currency']})")
    return account

def make_campaign(account, group_name):
    return account.create_campaign(params={
        "name": group_name,
        "objective": CAMPAIGN_OBJ,
        "status": Campaign.Status.paused,
        "special_ad_categories": [],
        "campaign_budget_optimization": True,
        "daily_budget": CAMPAIGN_BUDGET,
    })["id"]

def make_adset(account, campaign_id, group_name):
    return account.create_ad_set(params={
        "name": f"{group_name}_광고세트",
        "campaign_id": campaign_id,
        "status": AdSet.Status.paused,
        "optimization_goal": "OFFSITE_CONVERSIONS",
        "billing_event": "IMPRESSIONS",
        "bid_strategy": "LOWEST_COST_WITHOUT_CAP",
        "targeting": {
            "geo_locations": {"countries": TARGET_COUNTRIES},
            "age_min": AGE_MIN, "age_max": AGE_MAX,
        },
        "targeting_optimization": "EXPANSION_ALL",
        "promoted_object": {"pixel_id": PIXEL_ID, "custom_event_type": CONVERSION_EVENT},
    })["id"]

def upload_img(path):
    img = AdImage(parent_id=AD_ACCOUNT_ID)
    img[AdImage.Field.filename] = path
    img.remote_create()
    return img[AdImage.Field.hash]

def upload_vid(path):
    vid = AdVideo(parent_id=AD_ACCOUNT_ID)
    vid[AdVideo.Field.filepath] = path
    vid.remote_create()
    vid_id = vid["id"]
    time.sleep(10)
    return vid_id

def make_creative(account, ad_data, media_type, media_ref):
    story = {"page_id": PAGE_ID}
    if INSTAGRAM_ID:
        story["instagram_actor_id"] = INSTAGRAM_ID
    
    link = ad_data.get("랜딩 URL (link_url)", "")
    pt = ad_data.get("본문 텍스트 (primary_text)", "")
    hl = ad_data.get("헤드라인 (headline)", "")
    desc = ad_data.get("설명 (description)", "")
    cta = ad_data.get("CTA 버튼", "LEARN_MORE") or "LEARN_MORE"
    
    if media_type == "image":
        story["link_data"] = {
            "image_hash": media_ref, "link": link,
            "message": pt, "name": hl, "description": desc,
            "call_to_action": {"type": cta, "value": {"link": link}},
        }
    else:
        story["video_data"] = {
            "video_id": media_ref,
            "message": pt, "title": hl, "link_description": desc,
            "call_to_action": {"type": cta, "value": {"link": link}},
        }
    
    return account.create_ad_creative(params={
        "name": f"[크리에이티브] {ad_data['광고명']}_{media_type}",
        "object_story_spec": story,
        "degrees_of_freedom_spec": {
            "creative_features_spec": {"standard_enhancements": {"enroll_status": "OPT_IN"}},
        },
    })["id"]

def make_ad(account, adset_id, creative_id, name, suffix=""):
    return account.create_ad(params={
        "name": f"{name}{suffix}",
        "adset_id": adset_id,
        "creative": {"creative_id": creative_id},
        "status": Ad.Status.paused,
        "tracking_specs": [{"action.type": ["offsite_conversion"], "fb_pixel": [PIXEL_ID]}],
    })["id"]

print("✅ 함수 로드 완료")

---
## STEP 6: 광고 생성 실행 🚀

⚠️ **STEP 3~4 확인 후 실행하세요!**

In [ ]:
# 🚀 실제 광고 생성
print("=" * 70)
print("  🚀 Facebook Advantage+ 대량 광고 생성")
print("=" * 70)

account = init_api()
print()

# 카피 입력된 광고만 필터
valid_ads = [
    r for r in ad_list 
    if str(r.get("본문 텍스트\n(primary_text)","") or "").strip()
    and str(r.get("헤드라인\n(headline)","") or "").strip()
    and str(r.get("랜딩 URL\n(link_url)","") or "").strip()
]

print(f"📊 {len(valid_ads)}개 광고 처리 예정")
print()

# 광고그룹별로 묶기
groups = defaultdict(list)
for r in valid_ads:
    groups[r["광고그룹명"]].append(r)

results = []
campaign_cache = {}

for group_name, group_ads in groups.items():
    print(f"\n{"="*60}")
    print(f"📁 광고그룹: {group_name}")
    print(f"{"="*60}")
    
    # 캠페인 (광고그룹명 = 캠페인명)
    if group_name not in campaign_cache:
        campaign = make_campaign(account, group_name)
        campaign_cache[group_name] = campaign
    else:
        campaign = campaign_cache[group_name]
    
    # 광고 세트
    adset = make_adset(account, campaign, group_name)
    
    for ad_row in group_ads:
        ad_name = ad_row["광고명"]
        folder  = str(ad_row.get("소재 폴더", "") or "")
        pt      = str(ad_row.get("본문 텍스트\n(primary_text)", "") or "")
        hl      = str(ad_row.get("헤드라인\n(headline)", "") or "")
        desc    = str(ad_row.get("설명\n(description)", "") or "")
        url     = str(ad_row.get("랜딩 URL\n(link_url)", "") or "")
        cta     = str(ad_row.get("CTA 버튼", "") or "LEARN_MORE")
        
        images, videos = collect_media(ADS_BASE_FOLDER, folder)
        print(f"\n  🎨 {ad_name}: 이미지 {len(images)}개 + 영상 {len(videos)}개")
        
        # 이미지 광고
        for i, img_path in enumerate(images):
            img_hash = upload_img(account, img_path)
            creative = make_creative(
                account, f"{ad_name}_img{i+1}",
                img_hash=img_hash,
                primary_text=pt, headline=hl,
                description=desc, link_url=url, cta=cta
            )
            ad = make_ad(account, adset, creative, f"{ad_name}_img{i+1}")
            results.append({"type":"img", "ad_name":ad_name, "ad_id":ad["id"]})
            time.sleep(1)
        
        # 영상 광고
        for i, vid_path in enumerate(videos):
            vid_id = upload_vid(account, vid_path)
            creative = make_creative(
                account, f"{ad_name}_vid{i+1}",
                video_id=vid_id,
                primary_text=pt, headline=hl,
                description=desc, link_url=url, cta=cta
            )
            ad = make_ad(account, adset, creative, f"{ad_name}_vid{i+1}")
            results.append({"type":"vid", "ad_name":ad_name, "ad_id":ad["id"]})
            time.sleep(2)

print(f"\n\n{"="*70}")
print(f"  ✅ 완료! 총 {len(results)}개 광고 생성 (PAUSED 상태)")
print(f"{"="*70}")

# 결과 저장
result_path = Path(ADS_BASE_FOLDER) / "fb_results.json"
with open(result_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"📄 결과 저장: {result_path}")
